The idea of this notebook is to provide one plot showing that greedy faasta (combining Gale and Jingwei's ideas) is still slower than Condat for an analysis reconstruction problem with undecimated wavelets.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

# Third party import
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
from tqdm import tqdm_notebook

# Package import
from modopt.math.metrics import ssim
from modopt.opt.linear import Identity
from modopt.opt.proximity import SparseThreshold, LinearCompositionProx, LinearCompositionIterativeProx
from mri.numerics.fourier import FFT2, NFFT
from mri.numerics.gradient import GradAnalysis2
from mri.numerics.linear import Wavelet2, WaveletUD
from mri.numerics.reconstruct import sparse_rec_fista, sparse_rec_condatvu
from mri.numerics.utils import convert_mask_to_locations
import pysap
from pysap.data import get_sample_data

np.random.seed(0)


                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=0.18.0  - required | 1.3.0     installed
numpy          : >=1.11.0  - required | 1.16.3    installed
matplotlib     : >=2.0.0   - required | 3.1.0     installed
future         : >=0.16.0  - required | 0.17.1    installed
astropy        : ==2.0.8   - required | 2.0.8     installed
nibabel        : >=2.1.0   - required | 2.4.0     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar2   : >=3.34.3  - required | ?         installed
modopt         : >=1.1.5   - requi

#  Loading input data

In [2]:
image = get_sample_data("mri-slice-nifti")
original_image_data = np.copy(image.data)
image.data += np.random.randn(*image.shape) * 20.
image.data = image.data.astype(np.complex128)
mask = get_sample_data("mri-mask")

In [3]:
non_cartesian = True
sparkling = True

# Generate the kspace 
From the 2D brain slice and the acquistion mask, we generate the acquisition measurments, the observed kspace. We then reconstruct the zero order solution.

In [4]:
# potential path to sparkling trajectory
sparkling_traj_file_path = '../personal_experiments/2019-Mar-01_N512_nc34_ns3073_OS1_decim64_decay2_tau0.75_nrevol1/samples_SPARKLING_N512_nc34x3073_OS1.mat'

In [5]:
# Generate the subsampled kspace
if non_cartesian:
    if sparkling:
        kspace_loc = scipy.io.loadmat(sparkling_traj_file_path)['samples']
    else:
        kspace_loc = convert_mask_to_locations(mask.data)
    fourier_op = NFFT(samples=kspace_loc, shape=image.shape)
    kspace_data = fourier_op.op(image.data)

    # Zero order solution
    image_rec0 = pysap.Image(data=fourier_op.adj_op(kspace_data), metadata=image.metadata)
else:
    kspace_mask = np.fft.ifftshift(mask.data)
    kspace_loc = convert_mask_to_locations(kspace_mask)
    fourier_op = FFT2(samples=kspace_loc, shape=image.shape)
    kspace_data = fourier_op.op(image.data)

    # Zero order solution
    image_rec0 = pysap.Image(data=fourier_op.adj_op(kspace_data), metadata=image.metadata)

/volatile/home/Zaccharie/workspace/pysap-mri/mri/reconstruct/fourier.py:172: UserWarning: Samples will be normalized between [-0.5; 0.5[
  warnings.warn("Samples will be normalized between [-0.5; 0.5[")
/volatile/home/Zaccharie/workspace/pysap-mri/mri/reconstruct/utils.py:103: UserWarning: Frequency equal to 0.5 will be put in -0.5
  warnings.warn("Frequency equal to 0.5 will be put in -0.5")


# Operators

In [6]:
nb_scales = 4
wavelet_id = 2

In [7]:
linear_op = WaveletUD(
    nb_scale=nb_scales,
    wavelet_id=wavelet_id,
    coarse=False,
    set_norm=1.1,
)

gradient_op = GradAnalysis2(
    data=kspace_data,
    fourier_op=fourier_op)

prox_op_iterative = LinearCompositionIterativeProx(
    linear_op=linear_op,
    prox_op=SparseThreshold(Identity(), None, thresh_type="soft"),
    max_precision_level=150,
    solver_sigma=10.0,
)

prox_op_condat = SparseThreshold(linear_op, None, thresh_type="soft")

cost_op = None

# FAASTA optimization
We now want to refine the zero order solution using a FAASTA optimization.

In [8]:
def objective_cost(x):
    return data_fidelity(x) + sparsity(x)

def sparsity(x):
    return mu * np.sum(np.abs(linear_op.op(x)))

def data_fidelity(x):
    return 0.5 * np.linalg.norm(fourier_op.op(x) - kspace_data)**2

In [9]:
def nrmse(x):
    return np.linalg.norm(x - original_image_data) / np.mean(original_image_data)

In [10]:
metrics_ = {
    "cost": {"metric": objective_cost, "mapping": {"x_new": "x"}, "cst_kwargs": {}, "early_stopping": False},
    "nrmse": {"metric": nrmse, "mapping": {"x_new": "x"}, "cst_kwargs": {}, "early_stopping": False},
}

In [11]:
mu = 15.0

In [12]:
max_iter = 1000

In [13]:
# Start the FISTA reconstruction
cost_op = None

_, _, _, metrics_iterative = sparse_rec_fista(
    gradient_op=gradient_op,
    linear_op=Identity(),
    prox_op=prox_op_iterative,
    cost_op=cost_op,
    xi_restart=0.96,
    s_greedy=1.1,
    mu=mu,
    restart_strategy='greedy',
    pov='analysis',
    iterative_prox=True,
    max_nb_of_iter=max_iter,
    metrics=metrics_,
    metric_call_period=1,
    verbose=1,
    adaptative_precision=True,
    precision_increase_rate=1.01,
    initial_precision_level=5,
)


  _____             ____     _____      _
 |" ___|    ___    / __"| u |_ " _| U  /"\  u
U| |_  u   |_"_|  <\___ \/    | |    \/ _ \/
\|  _|/     | |    u___) |   /| |\   / ___ \\
 |_|      U/| |\u  |____/>> u |_|U  /_/   \_\\
 )(\\\,-.-,_|___|_,-.)(  (__)_// \\\_  \\\    >>
(__)(_/ \_)-' '-(_/(__)    (__) (__)(__)  (__)
    
 - mu:  15.0
 - lipschitz constant:  95.5108226538341
 - data:  (512, 512)
 - max iterations:  1000
 - image variable shape:  (512, 512)
 - alpha variable shape:  (512, 512)
----------------------------------------
Starting optimization...


100% (5 of 5) |##########################| Elapsed Time: 0:00:01 Time:  0:00:01
N/A% (0 of 1000) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--WARNING: Making input data immutable.
100% (5 of 5) |##########################| Elapsed Time: 0:00:01 Time:  0:00:01
  0% (1 of 1000) |                       | Elapsed Time: 0:00:02 ETA:   0:40:14WARNING: Making input data immutable.
100% (5 of 5) |##########################| Elapsed Time: 0:00:01 Time:  0:00:01
  0% (2 of 1000) |                       | Elapsed Time: 0:00:04 ETA:   0:40:30WARNING: Making input data immutable.
100% (5 of 5) |##########################| Elapsed Time: 0:00:01 Time:  0:00:01
  0% (3 of 1000) |                       | Elapsed Time: 0:00:07 ETA:   0:40:12WARNING: Making input data immutable.
100% (5 of 5) |##########################| Elapsed Time: 0:00:01 Time:  0:00:01
  0% (4 of 1000) |                       | Elapsed Time: 0:00:09 ETA:   0:39:43WARNING: Making input data immutable.
100% (5 of 5) |

  3% (39 of 1000) |                      | Elapsed Time: 0:01:31 ETA:   0:39:40WARNING: Making input data immutable.
100% (5 of 5) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1000 of 1000) |####################| Elapsed Time: 0:01:32 Time:  0:01:32


KeyboardInterrupt: 

In [ ]:
# Start the Condat reconstruction
cost_op = None
_, _, _, metrics_condat = sparse_rec_condatvu(
    gradient_op,
    linear_op,
    prox_op_condat,
    cost_op,
    std_est_method='dual',
    std_est=0.1,
    mu=mu,
    sigma=10,
    nb_of_reweights=0,
    max_nb_of_iter=max_iter,
    metrics=metrics_,
    metric_call_period=1,
    verbose=1,
)

In [ ]:
opt_metrics = {metric_name: metric_val['values'][-1] for metric_name, metric_val in metrics_condat.items()}

In [ ]:
metric_name = 'cost'
plt.figure(figsize=(9, 5))
plt.plot(
#     np.log10(np.abs(np.array(metrics_condat[metric_name]['values'] - opt_metrics[metric_name]))), 
    np.log10(np.abs(np.array(metrics_condat[metric_name]['values']))), 
    label='Condat', 
)
plt.plot(
#     np.log10(np.abs(np.array(metrics_iterative[metric_name]['values'] - opt_metrics[metric_name]))), 
    np.log10(np.abs(np.array(metrics_iterative[metric_name]['values']))), 
    label='greedy FAASTA', 
)
plt.legend()

In [ ]:
metrics_iterative['nrmse']['values'][-1]